In [1]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from utils.mylib import *
from d2l import torch as d2l

In [2]:
init_Seed()
device = get_device()

GPU ready!


In [3]:
# gg = torch.nn.Parameter(torch.ones(3, 2, requires_grad=True))
# N = torch.tensor([[0, 1], [0, 1], [0, 1]])

In [4]:
N = torch.load("./N20_100K.pt").data.to(device)
K = 20

In [5]:
class GNN(nn.Module):
    def __init__(self, n_users=1050, m_items=2050, n_factors=20):
        super(GNN, self).__init__()

        self.U = torch.nn.Parameter(torch.rand(n_users, n_factors, requires_grad=True))

        self.V = torch.nn.Parameter(torch.rand(m_items, n_factors, requires_grad=True))
        
        # self.E = nn.Sequential(nn.Linear(1000 * n_factors, n_factors), nn.ReLU(), nn.Linear(n_factors, n_factors))

        # self.E = torch.nn.Parameter(torch.rand(m_items, n_factors, requires_grad=True))

        self.U.data.uniform_(-0.05, 0.05)
        self.V.data.uniform_(-0.05, 0.05)
        
        self.criterion = nn.MSELoss(reduction='mean')
    
    def forward(self, user, item):
        neighbors = N[item]
        # print(neighbors.shape)
        return torch.einsum('ij, ij -> i', [self.U[user], self.V[neighbors].sum(dim=0)])
        # return torch.einsum('ij, ij -> i', [self.U[user], self.net(self.P[item])])

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        return self.criterion(pred, target)

In [6]:
def train(tr_set, dv_set, model, config):
    n_epochs = config['n_epochs']  # Maximum number of epochs
    batch_size = config['batch_size']

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)


    min_mse = 1000.
    loss_record = {'train': [], 'dev': []} 
    early_stop_cnt = 0
    epoch = 0

    while epoch < n_epochs:
        model.train()
        for X, y in tr_set:
            optimizer.zero_grad()    
            X, y = X.to(device), y.to(device)
            y_hat = model(X[:, 0], X[:, 1])

            mse_loss = model.cal_loss(y_hat, y)
            mse_loss.backward()
            
            optimizer.step()

            # loss_record['train'].append(mse_loss.detach().cpu().item())
            loss_record['train'].append(mse_loss.detach().cpu().item())

            # print("train_loss: {:.4f}".format(mse_loss.detach().cpu().item()))
            print("train_loss: {:.4f}".format(np.sqrt(mse_loss.detach().cpu().item())))

        scheduler.step()
        
        epoch += 1

        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            min_mse = dev_mse
            early_stop_cnt = 0
            print("Saving model (epoch = {:4d}  loss = {:.4f} )".format(epoch, np.sqrt(dev_mse)))
            # torch.save(model.state_dict(), config['save_path'])
        else:
            early_stop_cnt += 1
        
        loss_record['dev'].append(dev_mse)

        if early_stop_cnt > config['early_stop']:
            break

    print("Finish training after {} epochs".format(epoch))
    return min_mse, loss_record

In [7]:
               # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/           

config = {
    'n_epochs': 500,              # maximum number of epochs
    'batch_size': 5000,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.005,                # learning rate
        # 'weight_decay': 0.001
        # 'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 10,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth',  # your model will be saved here
    'D': 20
}

# **Load data and model**

In [8]:
ML100K_train = '../data/ML100K/ML100K_copy1_train.txt'
ML100K_test = '../data/ML100K/ML100K_copy1_test.txt'

ML1M_train = '../data/ML1M/ML1M_copy1_train.txt'
ML1M_test = '../data/ML1M/ML1M_copy1_test.txt'

tr_set = prep_dataloader(ML100K_train, 'train', config['batch_size'])
dv_set = prep_dataloader(ML100K_test, 'dev', config['batch_size'])
# tt_set = prep_dataloader("data/ML100K/ML100K_copy1_test.txt", 'test', config['batch_size'], target_only=target_only)

Max user: 943
Max item: 1682
Finished reading the train set of MoviesLen Dataset (60000 samples found, each dim = 2)
Max user: 943
Max item: 1679
Finished reading the dev set of MoviesLen Dataset (20000 samples found, each dim = 2)


In [9]:
model = GNN(n_factors=config['D']).to(device)
model_loss, model_loss_record = train(tr_set, dv_set, model, config)

torch.Size([5000, 20])


RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [5000, 20]->[5000, 20] [20, 20]->[20, 20]

# **Start Training!**

In [ ]:
plot_learning_curve(model_loss_record, title='MF model')

NameError: name 'model_loss_record' is not defined

In [22]:
gg = torch.tensor([[1, 2], [3, 4], [5, 6]])

In [23]:
nei = torch.tensor([[0, 1, 2], [2, 0, 1]])

In [24]:
gg[nei[0]].sum(dim=0)

tensor([ 9, 12])

In [25]:
gg = gg[nei.reshape(1, -1)].reshape(-1, 2)
gg

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [5, 6],
        [1, 2],
        [3, 4]])

In [31]:
torch.stack(gg.split(3, dim=0)).sum(dim=1)

tensor([[ 9, 12],
        [ 9, 12]])

In [ ]:
np.sqrt(model_loss)

0.9459027268389076

In [ ]:
np.sqrt(model_loss_record['dev'][-1])

0.9459075000997532

In [ ]:
# del model
# model = MF().to(device)
# ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
# model.load_state_dict(ckpt)
# # plot_pred(dv_set, model, device)  # Show prediction on the validation set
# dev(dv_set, model, device)